In [138]:
import pandas as pd
import numpy as np
from keras.utils import np_utils

In [139]:
df = pd.read_csv("iris.csv")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [140]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [141]:
features = list(df.columns[:4])

x = df[features]
print(x.head())

x = np.array(x)


y = list(df['species'])

y = np.array(y)

   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2


### Label encoding

The labels vector contains labels for the flowers in a single dimension. Using the LabelEncoder we convert that to 3 dimensions each  with 1 or 0 according to whether or not they fall into any of the 3 types

In [142]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y1 = encoder.fit_transform(y)
y = pd.get_dummies(y1).values

# Alternative for checking the head in ndarray
print(y[:5])

[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]]


### Train/Test Split

We us the train_test_split function in the model_selction module to split the data as follows:

 - 30% test data
 - 70% training data

In [143]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.30, random_state=0)

### Neural Network Architecture

Below we are going to define using keras a neural network model with the following layers

 - Layer 1: Input layer with 4 units (feature matrix has 4 dimensions) RELU activation
 - Layer 2: Inner layer with 5 units RELU activation
 - Layer 3: Output layer with 3 units SOFTMAX activation

This is Sequential model is created incrementally via the add() method. 

In [144]:
import keras

# Define model with keras_Squential
model = keras.Sequential()

# Using 4 units at input as the feature matrix has 4 dimensions
model.add(Dense(units = 4, activation = 'relu'))

# Inner layer defined with 5 units
model.add(Dense(units = 5, activation = 'relu'))

# Using 3 output units as the label matrix has 3 dimensions
model.add(Dense(units = 3, activation = 'softmax'))

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=200, batch_size=5, shuffle=True)


curacy: 0.4168
Epoch 6/200
21/21 [==============================] - 0s 652us/step - loss: 1.0203 - accuracy: 0.3537
Epoch 7/200
21/21 [==============================] - 0s 650us/step - loss: 1.0185 - accuracy: 0.3266
Epoch 8/200
21/21 [==============================] - 0s 649us/step - loss: 0.9893 - accuracy: 0.3954
Epoch 9/200
21/21 [==============================] - 0s 601us/step - loss: 0.9772 - accuracy: 0.4534
Epoch 10/200
21/21 [==============================] - 0s 700us/step - loss: 0.9549 - accuracy: 0.5903
Epoch 11/200
21/21 [==============================] - 0s 600us/step - loss: 0.8790 - accuracy: 0.7065
Epoch 12/200
21/21 [==============================] - 0s 650us/step - loss: 0.9440 - accuracy: 0.6521
Epoch 13/200
21/21 [==============================] - 0s 650us/step - loss: 0.9107 - accuracy: 0.7355
Epoch 14/200
21/21 [==============================] - 0s 700us/step - loss: 0.8805 - accuracy: 0.6377
Epoch 15/200
21/21 [==============================] - 0s 600us/step - l

In [145]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (5, 4)                    20        
_________________________________________________________________
dense_46 (Dense)             (5, 5)                    25        
_________________________________________________________________
dense_47 (Dense)             (5, 3)                    18        
Total params: 63
Trainable params: 63
Non-trainable params: 0
_________________________________________________________________


In [146]:
scores = model.evaluate(x_test, y_test)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

2/2 [==============================] - 0s 1ms/step - loss: 0.1231 - accuracy: 0.9778

Accuracy: 97.78%


### Testing predictions

The following statement will create a vector of all predictions for x_test matrix using the newly trained model

In [147]:
y_pred = model.predict(x_test)

### Prediction sample

The following shows the probabilities computed for the first row of the x_train matrix

In [148]:
x_test[1,:],y_pred[1]

(array([6. , 2.2, 4. , 1. ]),
 array([0.00956219, 0.96322066, 0.02721713], dtype=float32))

### Interpretation of prediction

There are 3 classes
 - Setosha(class 0)
 - Versicolor (class 1)
 - Virginica (class 2)

So we would like to pick a class based on the y_pred row picked above (0.003, 0.99, 0.004) which shows that the highest probability of classification is in favor of versicolor, i.e class 1

So to interpret the output layer we'll simply pick the highest probability and return the index as the classification, using numpy_argmax() function

In [149]:
y_pred_class = np.argmax(y_pred[1])
print(y_pred_class)

1


#### Save the model to the file system

Now we use keras to save the model to the file system for future use

In [150]:
model.save('model_file.iris')

INFO:tensorflow:Assets written to: model_file.iris\assets


## Prediction from saved model


Now we load the saved model from the file system and use it to run some predictions from saved data

In [151]:
import numpy as np 
from keras.models import load_model

iris_model = load_model('model_file.iris')

Create input values with which to test the model

In [152]:
data = np.array([[4.9, 3.0, 1.4, 0.9]])
print(data.shape)


(1, 4)


In [165]:
predictions_1 = iris_model.predict(data)

# Set 6 decimal positions inorder for the output probability to be human readable
np.set_printoptions(precision=6, suppress=True)

print(predictions_1)

print("\nPrediction 1 class : %i" % np.argmax(predictions_1))

[[0.966116 0.033884 0.      ]]

Prediction 1 class : 0


In [166]:
predictions_2 = iris_model.predict(np.array([[6,2,1.8,2]]))

print(predictions_2)

print("\nPrediction 2 class : %i" % np.argmax(predictions_2))

[[0.213423 0.786559 0.000018]]

Prediction 2 class : 1
